# Presentation

#### Goal: Predict the number of nights granted to a request (not to an individual)


#### Caracteristics of the model:
Accuracy:
- if used as a clearing tool removing overburden upfront, and thus only to get rid of obvious cases, the accuracy of such a tool could be its most important caracteristic.

Interpretability:
- could help families understand the decision (although not as important as in diseases predictions).
- can also highlight and thus control biases (racial, sex, age).
- since the tool would probably be used in combination with human selection, it could help save time by highlighting the main factors for each decision

#### Conclusion:
- a model easily interpretable could be prefered (tree).
- or a highly accurate model (less interpretable) could also be used upfront (NN).


#### Future improvements

Imputations:
- Build more robust, generalisable imputations (eg. impute future test samples with missing gender based on all easily interpretable categories group_composition_label and group_composition_id)

- Automate NaNs imputation for future test samples

- Reconstruct some NaNs by training models to predict the feature


Feature engineering

Datasets handling
- I made the choice to keep the train and test sets split. This is to prevent gaining insight from the test set while doing pre-processing, analysis and while training the model. To prevent duplicate code, I thus placed most of the inner workings in the class Analysis, which makes the reading less fluent. To improve, see how to better combine the visual aspect of Jupyter Notebooks, while maintaining code standards like DRY philosophy.


# Plan

I. Initialization

1. Import packages, classes, functions

2. Load databases (no join)

3. Instanciate object Analysis


II. Analyze I

1. Overview

2. Features: corr, dist, impact


III. Pre-process data

1. Impute NaNs

2. Remove outliers

3. Transform categorical features

4. Feature engineer


IV. Analyze II

1. Impact engineered features


V. Build Model

1. Benchmakrs

2. Rush pytorch NN

3. Simple model using principal components

4. Ensemble


VI. Predictions

1. Train and Predict

2. Hyperparameter tunning (split train set)


VII. Evaluate methods


# Initialization

## Import packages, classes, functions

In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.metrics import log_loss
from sklearn import tree

#os.chdir('/Users/Pro/Desktop/Git_Contests/Predictions/Emergency_housing/')
from cobratools import Analysis

In [2]:
# Define the test scorer
def competition_scorer(y_true, y_pred):
    return log_loss(y_true, y_pred, sample_weight=10**y_true)

In [3]:
# set to False to only apply data transformations
# (rather than derive the whole analysis)
ANALYZE_ON = True

# Set to True to visualize
PRINT_ON = True  # verbose
PLOT_ON = True

## Load data

Join datasets?

Since there are multiple requests by individuals and multiple individuals by request, the straightfoward approach would be to create columns for each individual' informations. This way, no information would be lost, but the curse of dimensionality is very near and the number of samples might be too low to extract useful information.

The chosen approach is rather to only keep the request dataset' columns, and feature engineer additional columns based on the individuals data, eg.:
- nb of past requests made by the same individual
- nb nights granted in past requests of the same individual(s)/group
- gender diversity of the group
- etc.

In [4]:
requests_train = pd.read_csv(filepath_or_buffer='data/requests_train.csv',
                            sep=',',
                            low_memory=False,
                            error_bad_lines=False)

requests_test = pd.read_csv(filepath_or_buffer='data/requests_test.csv',
                            sep=',',
                            low_memory=False,
                            error_bad_lines=False)

individuals_train = pd.read_csv(filepath_or_buffer='data/individuals_train.csv',
                                sep=',',
                                low_memory=False,
                                error_bad_lines=False)

individuals_test = pd.read_csv(filepath_or_buffer='data/individuals_test.csv',
                            sep=',',
                            low_memory=False,
                            error_bad_lines=False)

In [5]:
# Set index col as request id
requests_train.set_index('request_id', inplace=True)
requests_test.set_index('request_id', inplace=True)

## Instanciate object analysis

In [6]:
# Instanciate analysis object with request data
analyze_train = Analysis(requests_train)
analyze_test = Analysis(requests_test)

# Define properties

# - target
target = 'granted_number_of_nights' 
analyze_test.target = target
analyze_train.target = target
# - shape
analyze_train.m = analyze_train.df.shape[0]
analyze_test.m = analyze_test.df.shape[0]
analyze_train.n = analyze_train.df.shape[1]
analyze_test.n = analyze_test.df.shape[1]

# Analysis I

Overview

- Number of requests: 238191

- Number of individuals: 384133

- Number of features: 39

- Requests are made for 1.6 pers on average.


Principal components
- housing_situation_2_label: with value "emergency accomodation". High probability to get 1 or two nights.

--------------------


Analysis by features


A. housing_situation_id
- correlation target-housing_situation_id: -0.458581. Strong negative impact. Although the linear numerical relation of the housing_situation_id categories is in my opinion flawed, the strong correlation is explainable as the category with the smallest value "emergency accomodation" might be correlated with higher granted_number_of_nights than the rest of the categories, which happen to have higher housing_situation_id values.

- housing_situation_2_id: 0.283840. Strong positive impact. Same explanation as housing_situation_id.

B. pregnancy
- Pregnancy seems not to have a significant direct correlation with target

## Overview

In [332]:
analyze_train.describe(investigation_level=3)



Properties
----------


- Shape: (238191, 22)

- Features: ['animal_presence' 'answer_creation_date' 'child_situation'
 'child_to_come' 'district' 'granted_number_of_nights'
 'group_composition_id' 'group_composition_label' 'group_creation_date'
 'group_id' 'group_main_requester_id' 'group_type'
 'housing_situation_label' 'long_term_housing_request'
 'number_of_underage' 'request_backoffice_creator_id'
 'request_creation_date' 'requester_type' 'social_situation_id' 'town'
 'victim_of_violence' 'victim_of_violence_type']


Header
------


                                 animal_presence     answer_creation_date  \
request_id                                                                  
0000602749161524aedd04269957e5fc               f    2019-01-16 07:40:29.7   
0000c66f6bb24e930cba6a3872d3ab02               f  2019-02-14 17:13:12.018   
0000dbd253c096eab97cab0545915ab2               f  2019-03-24 19:24:04.327   
0001826bdd626262ec2bda3a1b908aac               f  2019-02-21 13:54:51

In [ ]:
analyze_train.visualize()

## Analysis by features

### housing_situation_2_label

- ±42% (160,061) indiv with 'housing_situation_label' == 'street'

- ±41% (156,496) indiv with 'housing_situation_label' == 'street' and 'housing_situation_2_label' == 'on the street'

- ±75% (289,870) individuals are "on the street"

- A majority of requests with the label "emergency accomodation" obtains 1 or 2 nights.

#### Distribution

In [ ]:
# ±42% (160,061) indiv with 'housing_situation_label' == 'street'
df_full[df_full['housing_situation_label'] == 'street']

# ±41% (156,496) indiv with 'housing_situation_label' == 'street' and 'housing_situation_2_label' == 'on the street'
df_full.query("housing_situation_label == 'street' and housing_situation_2_label == 'on the street'")

# ±75% (289,870) individuals are "on the street"
df_full['housing_situation_2_label'].value_counts()

In [ ]:
#### Impact on target
# Impact of feature on target
join_key = 'request_id'
target = 'granted_number_of_nights'
feature = 'housing_situation_2_label'
mask = df_train[feature] == 'emergency accomodation'

# Hist: drop duplicate requests (due to indiv data merged)
df_train[mask][[join_key, target]].drop_duplicates().hist()
plt.show()

### animal_presence

In [ ]:
feature = 'animal_presence'
mask = df_train[feature] == 't'
df_train[mask][[feature, target]]

# Pre-process data

Methodology:

- Clean-up request dataset

- Feature engineer using indiv dataset

## Impute NaNs

Methodology:
- inspect NaNs on train set

- if pattern detected, apply modifications on train and test sets

Observations:
- reverse engineering: the system seems to derive group_composition_id from group_composition_label, and both are then necessary linked/redondant => drop group_composition_label

- there can be multiple individuals by request, and multiple requests by individual

Further:
- Impute 14 pregnancy NaNs from child_to_come (not useful for the current objective)

### NaNs train summary

In [8]:
if ANALYZE_ON:
    # Get Na counts: by feature, by sample
    na_ft_train, na_sp_train = analyze_train.get_na_counts()
    na_ft_test, na_sp_test = analyze_test.get_na_counts()
    if PRINT_ON:
        print('Train: NaNs count by feature\n\n', na_ft_train[na_ft_train!=0])
        #print('\n\nTest: NaNs count by feature\n\n', na_ft_test[na_ft_test!=0])

Train: NaNs count by feature

 child_to_come                145947
housing_situation_label       16748
long_term_housing_request    165556
town                         159959
victim_of_violence_type      234175
dtype: int64


### Impute child_to_come NaNs

Observation
- There are 145947 NaNs for child_to_come on the train set (in request)

- There are only 14 NaNs for pregnancy in train set (in individuals)

Hypotheses
- Hyp: child_to_come is True if any indiv of the group is pregnant

Conclusion
- => Impute child_to_come from the pregnancy in the group of indiv of the request

In [12]:
# This takes ±3min
# Impute train set
analyze_train.impute_child_to_come(df_indiv=individuals_train)
# Impute test set
analyze_test.impute_child_to_come(df_indiv=individuals_test)

### Impute housing_situation_label

The following code cells are organized as follows for clarity:
- Imputation steps

- Analysis

Observations

A. Meta numbers
- ±7% (16,748) NaNs for housing_situation_label (request)

- 0 NaN for housing_situation_id (request)

- 0 NaN for housing_situation_label_2 (individuals)

- 0 NaN for housing_situation_id_2 (individuals)

- 21 housing_situation_label (request)

- 22 housing_situation_id (request)

B. Specific ratios
- ±10% (23,309) indiv with NaN at 'housing_situation_label'

- ±90% (21,185) of missing housing_situation_label are housing_situation_2_label "on the street" 

C. Analysis 1
- housing_situation_id is derived from housing_situation_label

- housing_sitaution_label NaNs have their specific id: 170

- => there are actually 16748 NaNs for housing_situation_id (request)

D. Analysis 2
- housing_situation_2_label can be very diverse when housing_situation_label is NaN (17 over 21 cat)

- => housing_situation_label NaNs are not produced in a specific housing_situation_2_label situation

E. Analysis 3
- all individuals of a same request share the same housing_situation_2_label

- => NaNs do not come from a problem of aggregating indiv data

Hypotheses
- Hyp: the housing situation should logically have an impact on the result

- Hyp: proba being on the street is significantly high to always impute NaN with 'street'

- Hyp: housing_ids are not sorted in any specific order from which a logic could be derived. Thus, having numerical ids is dangereous and could lead to misinterpretations by the model.

Conclusion
- => Impute housing_situation_label NaNs as 'street'

- => Drop housing_situation_id

- => one-hot encode housing_situation_label


Further improvements
- derive more sub-groups: when a group is not housing_situation_label_2 'on the street', impute the request housing_situation_label with its most often matched value

#### Impute NaNs with the single value 'street'

In [318]:
# Impute housing_situation_label NaNs as 'street'
analyze_train.df.loc[analyze_train.df['housing_situation_label'].isna(), 'housing_situation_label'] = 'street'
analyze_test.df.loc[analyze_test.df['housing_situation_label'].isna(), 'housing_situation_label'] = 'street'

# Drop housing_situation_id
analyze_train.df.drop('housing_situation_id', axis=1, inplace=True)
analyze_test.df.drop('housing_situation_id', axis=1, inplace=True)

# One-hot encoding is applied later-on

#### Get the mapping of housing_situation_id - housing_situation_label
- housing_situation_id is derived from housing_situation_label

- housing_sitaution_label NaNs have their specific id: 170

- => there are actually 16748 NaNs for housing_situation_id (request)


In [308]:
if ANALYZE_ON:
    # Get mapping housing_situation_id - housing_situation_label
    map_housing_id_label = analyze_train.df.loc[:, ['housing_situation_id', 'housing_situation_label']].drop_duplicates()

    # Sort and drop index for clarity
    map_housing_id_label = map_housing_id_label.sort_values(by='housing_situation_id')
    map_housing_id_label.reset_index(drop=True, inplace=True)
    map_housing_id_label

,housing_situation_id,housing_situation_label
0,10,emergency structure
1,20,stabilisation structure
2,30,inclusion structure
3,40,hotel paid by the household
4,50,hotel paid by the emergency centre
5,60,police station
6,70,parental home
7,80,personal or marital home
8,90,detoxification center
9,100,shelters


#### Analyze relation between housing_situation_label (request) and housing_situation_2_label (indiv)

Make a temporary dataframe with columns:

| request_id | gr_nb_nights | hous_id | hous_lab | indiv_id | hous_2id | hous2_lab

In [199]:
if ANALYZE_ON:
    # request [gr_nb_nights, hous_id, hous_lab]
    rq = analyze_train.df.loc[:, ['granted_number_of_nights', 'housing_situation_id', 'housing_situation_label']]

    # individuals [request_id, indiv_id, hous_2id, hous2_lab]
    ind = individuals_train.loc[:, ['request_id', 'individual_id', 'housing_situation_2_id', 'housing_situation_2_label']]
    ind.set_index('request_id', inplace=True)

    # Merge request and individuals datasets
    df_full = pd.merge(rq, ind, on='request_id')

Study figures

In [290]:
if ANALYZE_ON:
    # Nb of indivs (an indiv is considered a new indiv at each request)
    nb_indivs = df_full.shape[0]

    # ±6% (23,309) indiv with NaN at 'housing_situation_label'
    df_full[df_full['housing_situation_label'].isna()]

    # ±90% (21,185) of missing housing_situation_label are housing_situation_2_label "on the street" 
    df_temp = df_full[df_full["housing_situation_label"].isna()]
    df_temp['housing_situation_2_label'].value_counts()

on the street                                   21185
day center, social services, associations        1462
other                                             282
emergency accomodation                             82
accomodated by friends or other                    75
public instition (hospital, retirement home)       67
residential roaming                                40
hotel not paid by an emergency structure           28
reception center for asylum seeker                 24
accomodated by family                              18
emergency centre for asylum seeker                 12
winter plan center                                 11
medical centre                                      8
evicted from public housing rental                  6
private housing rental                              3
hotel paid by the emergency structure               2
public housing rental                               2
inclusion accomodation                              2
Name: housing_situation_2_la

Inspect individuals with housing_situation_2_label "on the street":

- they can have a very diverse housing_situation (17 categories over a total of 21)

- => no pattern to derive from this point of view

In [289]:
if ANALYZE_ON:
    q = df_full.query("housing_situation_label.isna() and housing_situation_2_label != 'on the street'")

    # Count number of categories for housing_situation_2_label
    n_cat_2 = len(q.loc[:, 'housing_situation_2_label'].unique())

    # Total categories for the feature
    n_cat_1 = len(analyze_train.get_col_uniques('housing_situation_label'))

Check if housing_situation_label NA exist when indiv within a group have different housing_situation_2

- no divergence

- => when group, all indiv have the same housing_situation_2_label

In [268]:
if ANALYZE_ON:
    # takes ±1min
    if False:
        # Get request_id along with its group size
        rq_id = df_full.index.value_counts()
        for i in range(70000):
            # Get ith rq_id
            rq_id_i = rq_id.index[0]

            # Observe if housing_situation_label same for all the group members
            n_uni_grp = len(df_full.loc[rq_id_i]['housing_situation_label'].unique())
            n_uni_indiv = len(df_full.loc[rq_id_i]['housing_situation_2_label'].unique())

            if n_uni_grp > 1 or n_uni_indiv > 1:
                print(i)
                print(n_uni_grp, n_uni_indiv)
                print("\n")

#### Further improvements: derive the most probable mapping

In [305]:
if ANALYZE_ON:
    # Get mapping housing_situation_label - housing_situation_2_label
    df_full_no_na = df_full[~df_full.housing_situation_label.isna()]
    map_housing_labels = df_full_no_na.loc[:, ['housing_situation_label', 'housing_situation_2_label']].drop_duplicates()

    # Sort and drop index for clarity
    map_housing_labels = map_housing_labels.sort_values(by='housing_situation_label')
    map_housing_labels.reset_index(drop=True, inplace=True)
    map_housing_labels

,housing_situation_label,housing_situation_2_label
0,accomodation by a third party,inclusion accomodation
1,accomodation by a third party,house share
2,accomodation by a third party,hotel not paid by an emergency structure
3,accomodation by a third party,reception center for asylum seeker
4,accomodation by a third party,medical centre
...,...,...
319,street,detention
320,street,evicted from private housing rental
321,street,medical centre
322,street,accomodated by family


### Impute long_term_housing_request NaNs

Nb NaNs: 165556

In [ ]:
# TODO

### Impute town NaNs

Nb NaNs: 159959

In [ ]:
# TODO

### Impute victim_of_violence_type NaNs

Nb NaNs: 234175

is NaN if victim_of_violence is 'f'


=> Replace these NaNs by say 0

In [ ]:
# TODO

analyze_train.df['gender'].groupby(analyze_train.df['pregnancy']).value_counts()

# Pattern with child_situation
analyze_train.df[target].groupby(analyze_train.df['pregnancy']).value_counts()

# Requests granted (nb_nights > 0)
granted = analyze_train.df[target] > 0

## Outliers

### Gender

- => Only females are possibly pregnant, thus 30 males have made a mistake

In [ ]:
analyze_train.df['gender'].groupby(analyze_train.df['pregnancy']).value_counts()
# TODO: Retrieve the individual ids, and correct for male -> pregnancy = 0

## Transform categorical features
Prepare data to feed models

- booleans: replace by (1, 0)

- 2 < cats < 11: one-hot encoding

- No transform on features with more than 11 categories

In [ ]:
# Pre-process columns:
# booleans: 't', 't' => True, False
# Categorical with few classes => one-hot encoding
bools_train, failed_train = analyze_train.transform_categories(target=analyze_train.target)
bools_test, failed_test = analyze_test.transform_categories(target=analyze_test.target)

# Preprocess specific cat columns
analyze_train.convert_to_bool(col='group_type',
                            true_val='group',
                            false_val='individual')

# Preprocess specific cat columns
analyze_test.convert_to_bool(col='group_type',
                            true_val='group',
                            false_val='individual')

# Export data
analyze_train.export_data('data/data_train_preprocessed.csv')
analyze_test.export_data('data/data_test_preprocessed.csv')

## Feature Engineering

- nb of indivs in the group

- nb of past requests by indivs forming the group of the request

- nb of past granted request by indivs forming //

### Nb of past requests by an individual

In [ ]:
# Get number of ALL requests of indiv with max n_requests
ind_id = df_full['individual_id'].value_counts().index[0]

# Watch request made by the same indiv
df_full[df_full['individual_id'] == ind_id]

# TODO: Past requests only

# Analysis II
(df_train)

Analysis focused on impact of one-hot encoded variables and new engineered features

In [ ]:
# TODO

# Predict

## Build models

### Benchmarks

In [ ]:
# Random uniform train/test
random_preds_train = np.random.uniform(size=(requests_train.shape[0], 4))
random_preds_test = np.random.uniform(size=(requests_test.shape[0], 4))

# Dumb (always pred 3)
dumb_preds_train = np.zeros((requests_train.shape[0], 4))
dumb_preds_test = np.zeros((requests_test.shape[0], 4))
# Set 10% pred everywhere (if not, log penalyzes hardly)
dumb_preds_train[:,:] = .01
dumb_preds_test[:,:] = .01
# Set 20% pred on class 3
dumb_preds_train[:,2] = .02
dumb_preds_test[:,2] = .02

### Univariate predictions

We observed a significant (negative) correlation of housing_situation_id with granted_number_of_nights, let's train a univariate model


In [ ]:
# Set model' parameters
clf = tree.DecisionTreeClassifier(
    criterion='gini',
    splitter='best',
    max_depth=2,
    min_samples_split=2,
    min_samples_leaf=1,
    max_features=None,
    max_leaf_nodes=None,
    class_weight=None)

# Build train/test datasets with housing_situation_id only
X_train = np.array(requests_train['housing_situation_id']).reshape(-1, 1)
X_test = np.array(requests_test['housing_situation_id'].values).reshape(-1, 1)
Y_train = requests_train.granted_number_of_nights.values
Y_test = requests_test.granted_number_of_nights.values

# Transform categorical target into a one-hot vector
Y_train_onehot = to_onehot(Y_train)
Y_test_onehot = to_onehot(Y_test)

# Train the model
clf = clf.fit(X_train, Y_train_onehot)

# Yield train/test predictions
preds_train_tree_univar = clf.predict(X_train)
preds_test_tree_univar = clf.predict(X_test)

# Fill predictions to .2 elsewhere
preds_train_tree_univar[preds_train_tree_univar == 0] = .2
preds_test_tree_univar[preds_test_tree_univar == 0] = .2

# Evaluate train/test
score_train = competition_scorer(Y_train, preds_train_tree_univar)
score_test = competition_scorer(Y_test, preds_test_tree_univar)

# Display results
print(f'train score: {score_train:.2f}')
print(f'test score: {score_test:.2f}')

In [ ]:
probas = clf.predict_proba(X_train)
v0 = probas[0].max(1)
v1 = probas[1].max(1)
v2 = probas[2].max(1)
v3 = probas[3].max(1)

## Evaluate models

In [ ]:
y_true_test = requests_test.granted_number_of_nights.values

# Evaluate benchmarks
random_score_test = competition_scorer(y_true_test, random_preds_test)
dumb_score_test = competition_scorer(y_true_test, dumb_preds_test)

# Display results
print(f'test score random: {random_score_test:.2f}')
print(f'test score dumb: {dumb_score_test:.2f}')

### Train set

In [ ]:
y_true_train = requests_train.granted_number_of_nights.values

# Evaluate benchmarks
random_score_train = competition_scorer(y_true_train, random_preds_train)
dumb_score_train = competition_scorer(y_true_train, dumb_preds_train)

# Display results
print(f'train score random: {random_score_train:.2f}')
print(f'train score dumb: {dumb_score_train:.2f}')

### Test set

## Interpret models

In [ ]:
# Tree
fn = ['housing_situation_id']
cn = ['0', '1', '2', '3']

tree.plot_tree(clf,
               feature_names = fn, 
               class_names=cn,
               filled = True)

requests_train['housing_situation_id'].hist()